# Python学习笔记

## 第17章 作用域

### 变量的作用域

Python中的变量名解析遵从`LEGB`原则，即：

* 当在函数中使用未认证的变量名时，Python搜索4个作用域：本地作用域（L），之后是上一层结构中的`def`或`lambda`的本地作用域（E），之后是全局作用域（G），最后是内置作用域（B）并且在第一处能够找到这个变量名的地方停止下来。如果变量名在这次搜索中没有找到，Python会报错。

* 当在函数中给一变量名赋值时（而不是在一个表达式中引用），Python总是创建或改变本地作用域的变量名，除非它已经在那个函数中声明为全局变量。

* 当在函数之外给一个变量名赋值（也就是，在一个模块文件的顶层，或者是在交互模式下），本地作用域与全局作用域（这个模块的命名空间）是相通的。

In [10]:
#以下是一个全局作用域的例子
x = 99#一个全局变量
def func(y):
    z = x + y#因为x没有在本地作用域中声明，因此就引用在全局作用域中的x
    return z
func(1)

100

在函数中，由于`LEGB`法则，函数可以引用直接使用本地变量，也可以使用上层嵌套函数的变量，更可以直接使用全局变量，但是这种变量的使用仅仅是值的引用，如果下层作用域要改变上层作用域中定义的变量，则会报错或者直接在同层定义域中声明另一个变量，举例来说如下：

In [7]:
def modify_x():
    '''
    这里虽然尝试将x指向另一个值，但是这样的写法会让python编译器直接解释为声明一个本地变量，
    该变量的作用范围仅仅在这个函数中，一旦函数结束这个本地x就会被释放，不会对全局的x产生影响。
    '''
    x = 100
modify_x()
print(x)

99


In [20]:
def modify_x_1():
    '''
    这段程序则不使用赋值式的改变，而是直接让x加一，但是这样的操作如程序输出是会报错的。这就是上面说的：
    下层作用域（函数中的作用域）想要改变上层作用域（全局作用域）会导致失败。
    '''
    x += 1
modify_x_1()
print(x)

UnboundLocalError: local variable 'x' referenced before assignment

需要特别说明的是：如果是在A模块中声明了全局变量`x`，那么在B模块中只要`import A`模块，就可以使用`A.x=xxx`的形式改变A模块中的全局值，但是这样的做法是非常不建议的，因为这会大大降低程序的可读性。

### `global`关键字

`global`关键字的作用在于显式地引用全局变量，因为我们知道Python中对于变量的搜索遵从一套`LEGB`原则，思考下面一种情况：

In [12]:
def show_x():
    x = 150
    def show_x_internal():
        print(x)#这里会直接使用外层函数定义的本地变量x，而不会使用全局变量x
    show_x_internal()
show_x()

150


上面的代码展示了这样一种情况：外层函数通过赋值语句定义了一个本地变量`x`，内存嵌套函数想要引用`x`的值，但是由于`LEGB`法则，程序就会直接使用外层函数定义的本地变量，而不是全局变量。如果我们一定想要引用全局变量，那么加上`global`关键字是一个好的选择：

In [13]:
def show_x_1():
    x = 150
    def show_x_internal_1():
        global x#这里声明了使用全局变量x，而不是外层函数定义的本地变量x
        print(x)
    show_x_internal_1()
show_x_1()

99


`global`关键字除了显式指定引用全局变量外，还有一个功能就是可以改变全局变量的值，也可以在函数中动态声明全局变量，举例如下：

In [14]:
def init_var():
    global x
    x = 145
    global y, z##原本没有y和z两个全局变量，在这里可以使用gloabl关键字声明
    y = z = 56
init_var()
print(x, y, z)

145 56 56


如上面的代码所示：在函数内部使用`global`关键字，既可以改变全局变量值，也可以声明新的全局变量值，总结来说，`global`关键字的作用总结成为以下三点：

1. 显式引用全局变量，不论上层作用域中是否定义了同名变量，只要使用`global`关键字就可以引用全局变量。

2. 改变全局变量的值：默认情况下，函数内部只能引用全局变量的值而不能改变，如果使用了`global`关键字则可以对其值进行改变。

3. 可以使用`global`关键字声明新的全局变量值。

### `nonlocal`关键字

`nonlocal`和`global`关键字的作用十分相近：他们都可以针对上层作用域的值进行修改，不同的是`nonlocal`关键字只作用于嵌套函数的外层函数，与全局作用域无关且如果外层函数中没有定义这个变量就会报错而不会继续到全局作用域中寻找（使用`global`关键字引用一个变量时，如果该变量在全局作用域中没有创建，那么就会自动创建一个，但是`nonlocal`关键字与此不同，如果引用的变量在外层函数中没有创建，就会直接报错而不会创建）；

In [16]:
def nonlocal_test():
    x = 154
    def inner():
        '''
        nonlocal关键字只能在内层函数中使用，且引用的变量一定要在外层函数中定义过，否则会报错
        '''
        nonlocal x
        print(x)
    inner()
nonlocal_test()

154


In [19]:
def non_local_error():
    def inner():
        nonlocal x#只会在外层函数中寻找这个值，若没有找到就会报错而不会继续到上层作用域（全局作用域）中寻找
        print(x)
    inner()
non_local_error()

SyntaxError: no binding for nonlocal 'x' found (<ipython-input-19-11128f71c88c>, line 3)

如上面代码所示：在内部函数中使用`nonlocal`关键字引用的`x`变量虽然在全局变量中存在，但是`nonlocal`只在外层函数中寻找该变量，显然是找不到的，因此报错。

除此之外，`nonlocal`关键字还能用于修改外层函数中本地变量的值，如果没有使用`nonlocal`关键字则只能引用外层函数的值而不能去修改：

In [29]:
def modify_nonlocal():
    a = 100
    b = 150
    c= 200
    def inner():
        nonlocal a#这里使用nonlocal关键字引用了外层函数的a，这样就可以更改a的值
        a = 111#将a的值更改为111，外层函数的a值也改变了
        b = 130#这里只是在内层函数中又创建了一个b的本地变量而没有改变外层函数中的b
        print("a=%d" % (a), "b=%d" % (b), "c=%d" % (c))#内层函数可以直接使用外层函数的c的值却不能改变它的值
    inner()
    print("a=%d" % (a), "b=%d" % (b), "c=%d" % (c))#只有a的值被改变了。
modify_nonlocal()

a=111 b=130 c=200
a=111 b=150 c=200


如上面代码所示：只有使用`nonlocal`关键字的变量`a`的值被改变了，其余的两个变量只能引用他们的值却不能改变。总结来说，`nonlocal`关键字的作用如下：

1. 显式引用外层函数的变量：若外层函数中没有定义这个变量则会报错而不会创建一个或者是到全局作用域中寻找。

2. 改变外层函数中相应变量的值：只有使用了`nonlocal`关键字的变量才能够被改变值，否则只能引用而不能改变。

### 函数闭包

函数闭包一般来说也可以理解成一类变量作用域问题，举例如下：

In [41]:
def outer():
    N = 2
    def inner(m):
        return m ** N#内层函数将记录下这个值
    return inner
f = outer()
print(f(2))
print(f(4))
print(f(6))

4
16
36


如上面所示：这是一个工厂函数，该函数返回一个嵌套内层函数，该内层函数中使用了外层函数的值，这个值可以被内层函数记录下来，即使外层函数的调用已经结束该值依然存在。

在循环结构中，这种闭包的又变的有些不同：

In [34]:
def loop_outer():
    acts = []
    for i in range(5):
        '''
        按照预想：这里的每个lambda表达式会使用从0到4不同的i值作为指数，
        但是现实是全部的lambda都只会使用最后一个i值也就是4作为指数
        '''
        acts.append(lambda x : x ** i)
    return acts
lst = loop_outer()
print(lst[0](2))
print(lst[2](2))
print(lst[4](2))

16
16
16


如上面的代码所示：所有的lambda表达式都会使用最后一个`i`作为指数，这是由于lambda看做是一个内层函数在这里还是引用外层函数的`i`值，而`i`值现在变成了4，因此所有引用了`i`的都变成了4。要解决这个问题可以直接把`i`作为参数传入到内存函数中，这样内层函数中就有一个`i`的拷贝，这样就不会有上述问题了：

In [39]:
def loop_outer_1():
    acts = []
    for i in range(5):
        acts.append(lambda x, i=i: x ** i)
    return acts
lst = loop_outer_1()
print(lst[0](2))
print(lst[2](2))
print(lst[4](2))

1
4
16


如上所示：问题已经得到了解决。函数的闭包一类非常大的问题，这里只是做了简单的探讨，还有非常多的内容需要去理解。

## 第18章 参数

在Python中参数传递有以下原则：

* 不可变参数是以值来传递的：比如说字符串，数字这样的不可变类型传递的是它们的值。

* 可变参数是以“指针”形式传递的：比如说列表，字典等类型传递的是指针，这样在函数内部对他们的修改就能反映到函数外。

python支持可变参数列表。通常来说python支持按位置传递参数，支持关键字-值型的参数传递，同时也支持打包解包式的参数传递模式。无论是在函数声明还是在调用时参数的传入顺序都是十分重要的，通常在函数调用和声明时参数的顺序规定是相同的，都是按照如下顺序：位置的参数、关键字-值型参数、`*name`型参数和`**dict`型参数来传入。但是由于`keyword-only`型参数的存在使得函数在声明时可以在`*name`和`**dict`之间加入关键字-值型的参数，当然这是可选的。

In [64]:
def parameter_func(a, b, c, d=5, e=6, *args, p=101, **kargs):
    print("a={0}, b={1}, c={2}, d={3}, e={4}, p={5}".format(a, b, c, d, e, p))
    print("args={0}".format(args))
    print("kargs={0}".format(kargs))
parameter_func(1, 2, 3, 9, p=105, *(4,5,6,7,8,9), **{"name":"David", "age":15})

a=1, b=2, c=3, d=9, e=4, p=105
args=(5, 6, 7, 8, 9)
kargs={'age': 15, 'name': 'David'}


在使用函数参数列表的时候一定要谨慎，否则就会发生错误，比如说以下函数调用就是错误的：

In [56]:
parameter_func(1, 2, 3, 9, d=5, p=200, *(4,5,6,7,8,9), **{"name":"David", "age":15})

TypeError: parameter_func() got multiple values for argument 'd'

错误提示是说给变量`d`赋了多个值，这该怎么理解呢？首先我们应该了解在函数定义时`a=xxx`这样的写法只是表明变量`a`有一个默认值，如果没有参数传入那么他的值就是`xxx`，这在本质上和普通的参数声明是没有任何区别的，也就是说这样声明的参数依然可以通过位置来传入。接下来我们看这个错误的函数调用，我们将它解包成为简单易懂的函数调用：

```python
parameter_func(1, 2, 3, 9, 4, 5, 6, 7, 8, 9, d=5, p=200, name="David", age=15)
```

这样就明显的多了：函数调用时首先在参数列表的第4个位置给`d`赋了一个9，接着又通过`d=5`的形式给`d`赋了一个5，自然就会引发报错。

从上面的例子可以看出：无论在函数声明时是否声明了`keyword-only`型参数，函数在调用时都应遵循：位置的参数、关键字-值型参数、`*name`型参数和`**dict`型参数。

函数在调用和声明时都支持`*name`和`**dict`这一语法。在函数声明时`*name`代表的是元组，列表这类的数据结构，`**dict`代表的是字典这样的数据结构，函数在调用时也可以使用上述的两种`*`语法。举例来说有函数调用如
`func(*(1,2,3,4,5), **{'a':4, 'b':5})`，这样的写法等效于`func(1,2,3,4,5,a=4,b=5)`，这就是python中参数的解包。

而函数声明时的`*name`和`**dict`在使用上有两种模式：

* 一种是单纯只使用`*name`与`**dict`模式。

* 另外一种是与位置型参数和关键字-值型参数混合的模式。

前者的使用模式下可以在`*name`和`**dict`之间加入`keyword-only`型参数，这样单纯地作为传入可变参数来使用，`keyword-only`型参数起传递某些配置选项的作用。如果是第二种模式使用，那么`*name`和`**dict`的作用就在于函数收集完位置型参数和关键字-值型参数后多出来的这些参数的收集器，其中多出来的位置型参数被`*name`收集，多出来的关键字-值型参数被`**dict`收集起来。

其实两种模式是可用一套理论解释清楚的，这里之所以用两种理论是为了更清晰一些。试想在第一种模式下没有位置型参数和关键字-值型参数（不包含`keyword-only`型参数的情况下），函数在调用时传入的参数可看成全部是多出来的参数，那么位置型参数被`*name`收集，关键字-值型参数被`**dict`收集。

## 第6部分 类和OOP

### 多继承和类的搜索路径

Python作为一种面向对象的语言，也拥有面向对象编程的特点，在这里总结Python面向对象的知识点：

由于单纯的类十分简单，就不多说了，直接从方法重载，多继承开始说起。首先介绍多继承的知识点：Python中允许多继承，即一个类可以使多个类的子类。通过继承，类和类之间就能形成一个继承树。在Python3.0中的类均已改成新式类了，其具体的搜索规则是：首先搜索当前类自身，如果没有找到则按照继承列表从左至右，以广度优先遍历规则搜索。一旦找到调用的类方法或**类属性**就立即返回结果并停止搜索。

In [7]:
class superClass:
    age = 15
    def __init__(self):
        self.name = "Ann"
    def display(self):
        print("This is superClass")
    def __str__(self):
        return "This is superClass"
class FirstClass(superClass):
    def __init__(self):
        superClass.__init__(self)
        self.name = "David"
    def __str__(self):
        return "This is FirstClass"
class SecClass:
    age = 29
    def __init__(self):
        self.name = "Smith"
    def display(self):
        print("This is in SecClass")
    def __str__(self):
        return "This is SecClass"

class subClass(FirstClass, SecClass):
    def __init__(self):
        FirstClass.__init__(self)
        SecClass.__init__(self)
    def __str__(self):
        return "This is subClass"

obj = subClass()
obj.display()
print(obj.name)
print(obj.age)
print(obj)

This is superClass
Smith
15
This is subClass


观察以上代码可以发现：`subClass`使用了多继承，且其父类`FirstClass`又是`superClass`的子类。由程序的输出可以发现，当`subClass`调用`display`方法时，python虚拟机会首先在`FirstClass`中搜索，没有发现的话会继续在其父类`superClass`中搜索，一旦找到就返回找到的`display`方法，且停止搜索，而不会在`secClass`中继续搜索。

但是在打印`name`的时候我们发现打印出来的是`SecClass`的`name`属性，这是由于累继承实际上是类对象的继承，但是`name`是实例的属性，因此是不会带入继承类树中的。在`subClass`中的构造函数中分别先后调用了`FirstClass`和`SecClass`的构造函数，并且传入了自己的实例，由于`SecClass.__init__(self)`调用的最晚，因此`name`的值就被覆盖成最新的了。

接着打印`age`属性，该属性是一个类属性，是类对象的一半部分，因此会被带入到累继承中，python虚拟机首先会检查`subClass`中是否有`age`这个属性，没有的话就会一句继承列表从左到右且深度优先遍历的顺序寻找`age`属性。

最后打印对象，这里使用了`__str__`运算符重载，同时也说明了在累继承中的方法重载规则：只要Python虚拟机在当前类中找到了调用的方法`__str__`就不会继续搜索了。

**和大多数继承相同的是：子类要负责父类的构造，重载方法时最好调用原方法。然而在具体操作时一定不要使用`self.xxx()`的形式，因为这样可能会引发死循环**

类和实例还可以通过`shelve`和`pickle`工具进行持久化，具体见书676页。

总的来说，Python3.0中类的知识点归结于以下几点：

1. 所有的类都是类对象，跟实例相似，类对象也可以定义属性。在继承操作时实际上是继承类对象，因此类的属性和方法都会得到继承，而实例属性却不会

2. 当实例引用一个属性或方法时首先在当前类中寻找，然后依照继承列表中从左至右，广度优先遍历的顺序寻找。一旦找到就停止搜索。

3. 在实例和类对象上使用`=`运算符进行赋值操作时并不会触发搜索操作，而是会直接在调用实例上增加实例对象，若实例中本来就有这个属性，就更新它。


### 类接口技术

这个和元类有关，具体的原理还不清楚，代码如下：

In [10]:
from abc import ABCMeta, abstractmethod
class super(metaclass=ABCMeta):
    def delgate(self):
        self.action()
    @abstractmethod
    def action(self):
        pass
class sub(super):
    def action(self):
        print("This is sub")
obj = sub()
obj.delgate()

This is sub


主要注意的要点如下：

1. 在接口类上面要声明`metaclass=ABCMeta`。
2. 具体的接口方法上要使用`@abstractmethod`装饰器。

### 运算符重载

这一届主要是技巧性的东西，需要的时候可以再看细节，主要需要了解`__getattr__`和`__setattr__`两个方法，总结如下：

1. `__getattr__`将用于未定义的属性引用，例如`a.name`将引用`name`的值，但是如果`name`在实例和类以及其父类中都没有定义，则就会触发`__getattr__`方法，否则将不会。需要说明的是：不仅仅是属性值，调用未定义的方法也会触发`__getattr__`。

2. `__setattr__`将拦截任何赋值操作，不论赋值操作设计的变量是否定义。

3. 在`__getattr__`和`__setattr__`中都要使用`self.__dict__`进行操作，千万不能针对当前的`self`使用`.`运算符或`setattr`和`getattr`，否则会引发死循环。

### python中的多态

面向对象编程的三大特性之一——多态是被必须要被实现的。在python中依然会有体现。但是python的多态支持并不如c++或者java中那么健全，显得有些单调。python中的多态通常是在`X.method()`的调用中，method的意义取决于X的类型（类）。但是在其他的面向对象的语言中，若一个类中有多个同名方法（但参数列表或返回值不同），也能根据调用实现多态，举例来说就是如此：

In [1]:
class poly:
    def method(self, a, b):
        print("this is method1")
    def method(self, a, b, c):#这样的代码是可以被执行的，但是会覆盖掉第一个method方法
        print("this is method2")
p = poly()
p.method(1,2,3)
p.method(1,2)

this is method2


TypeError: method() missing 1 required positional argument: 'c'

如上所示：如果一个类中定义了多个同名方法，即使他们的参数列表是不同的，但是依然前面的方法会被后面的同名方法覆盖掉，这是python多态中一个比较弱的地方。

### 私有属性

python中不支持私有属性定义，换句话说python中的任何变量都可以被子类看见，但是可以通过一些小trick来实现私有属性定义：

1. 在变量名前面加`_`来提醒程序员，这是一个私有变量，是不能够被修改的，虽然这样的变量依然可以被访问和修改，就像正常的变量一样。

2. 通过运算符重载`__getattr__`和`__setattr__`来拦截属性访问，实现私有属性保护。

3. 通过变量名压缩的形式：在需要注明为私有的变量前加上`__`双下划线，这样该变量的名称就会自动变为`_类名__变量名`的形式，且只能在定义的类内部进行访问，因为如果他的子类也使用双下划线的形式访问该变量，那么他访问额变量会自动填充为他自己的类名加变量名的形式，而类名会和变量初始声明的类名不同。

In [15]:
class A:
    __name = "David"
    def __init__(self):
        self.__age = 15
    def display(self):
        print(self.__age, A.__name)
class B(A):
    __name = "Smith"
    def __init__(self):
        A.__init__(self)
        self.__age = 25
b = B()
b.display()

15 David


如上面的代码所示：A的子类虽然在构造函数中重新对`__age`进行了赋值，但是丝毫没有影响和覆盖掉A中的定义，这就是变量名压缩带来的近似私有变量。让我们来举一个相反的例子论证：

In [17]:
class A:
    name = "David"
    def __init__(self):
        self.age = 15
    def display(self):
        print(self.age, A.name)
class B(A):
    name = "Smith"
    def __init__(self):
        A.__init__(self)
        self.age = 25
b = B()
b.display()

25 David


如上面代码所示：新的代码中去掉了`__`，这样就导致了B中变量对A中的覆盖，导致`display`方法的执行出现了不一样的结果。如果A是一个精心设计的接口，设计者可能定不希望接口中的方法使用的变量值被使用者素篡改。

### `__slots__`和`__dict__`关键字

在python中`__slots__`内置类型使用时比较少的，但是依然有必要了解:

1. `__slots__`是类属性而非实例属性，这就是说：`x=A() x.__slots__ = [...]`这样创建是不行的。

2. `__slots__`和`__dict__`是互斥的，也就是说，类中出现了`__slots__`就不能出现`__dict__`（当然，这是对于实例而言的，类对象中依然拥有`__dict__`属性）。这样的结果是：由拥有`__slots__`属性的类创建出的实例无法动态增加属性，只能使用`__slots__`中定义的属性，这是因为动态增加类型其实是对`__dict__`进行操作，而`__slots__`和`__dict__`互斥。

3. 可以在`__slots__`中定义`__dict__`属性来打破规则二的限制。

### 静态方法和类方法

1. 静态方法和类方法通常使用`@staticmethod`和`classmethod`来修饰，或者使用`staticmethod()`和`classmethod()`函数来包装。

2. 类方法无论采用实例调用还是类对象调用，均会自动传入类对象。

3. 静态方法可以看成是类中的普通函数，它不自动传入任何参数。

In [18]:
class staticAndClass:
    @staticmethod
    def func():
        print("This is static method")
    def func_1():
        print("This is also a static method")
    func_1 = staticmethod(func_1)
    @classmethod
    def func_2(cls):
        print("This is a class method: ", cls)
    def func_3(cls):
        print("This is also a class method: ", cls)
    func_3 = classmethod(func_3)
obj = staticAndClass()
staticAndClass.func()
staticAndClass.func_1()
staticAndClass.func_2()
staticAndClass.func_3()
obj.func_2()
obj.func_3()

This is static method
This is also a static method
This is a class method:  <class '__main__.staticAndClass'>
This is also a class method:  <class '__main__.staticAndClass'>
This is a class method:  <class '__main__.staticAndClass'>
This is also a class method:  <class '__main__.staticAndClass'>


### 管理属性

python中管理属性通常有以下几种办法：

* `__getattr__`和`__setattr__`方法，把未定义的属性获取和所有的属性赋值指向通用的处理器方法。 

* `__getattribute__`方法，把所有属性获取都指向Python2.6的新式类和Python3.0的所有类中的一个泛型处理器方法。· 

* 内置函数，把特定属性访问定位到get和set处理器函数，也叫做特性(property)

* 描述符协议，把特定属性访问定位到具有任意get和set处理器方法的类的实例。

#### 首先看`property`定义的属性：

In [21]:
class person:
    def __init__(self, name):
        self.__name = name
    def getName(self):
        print("This is getName")
        return self.__name
    def setName(self, value):
        print("This is setName")
        self.__name = value
    def delName(self):
        print("This is delName")
        del self.__name
    name = property(getName, setName, delName, "This is name")
p = person("David")
print(p.name)
p.name = "Smith"
del p.name

This is getName
David
This is setName
This is delName


当然，上面的代码也可以用以下代码来重写，只是换了一种方式：不是使用`property`内置函数。而是使用装饰器：

In [22]:
class person:
    def __init__(self, name):
        self.__name = name
    @property
    def name(self):
        print("This is getName")
        return self.__name
    @name.setter
    def name(self, value):
        print("This is setName")
        self.__name = value
    @name.deleter
    def name(self):
        print("This is delName")
        del self.__name
p = person("David")
print(p.name)
p.name = "Smith"
del p.name

This is getName
David
This is setName
This is delName


**注意：上面的代码需要特别小心方法的名字还有装饰器的名字不能写错**

#### 描述符

描述符其实也是一个类，用于拦截相应的属性调用，以下是描述符的一个定义：

In [1]:
class Descriptor:
    def __init__(self):#是可选的，也可以不定义构造函数
        pass
    def __get__(self, instance, owner):
        print("This is __get__: ", self, instance, owner)
        return instance._name
    def __set__(self, instance, value):
        print("This is __set__: ", self, instance, value)
        instance._name = value
    def __delete__(self, instance):
        print("This is __delete__: ", self, instance)
        del instance._name

参数说明：

1. `self`：描述符的对象实例。

2. `instance`：描述符所附加的客户类的实例。

3. `owner`：描述符所要附加到的类实例，这个通常不同去管他，一般情况下也用不到。（举例来说：假设描述器被用在了`A`类中，则第二点的参数是A的实例，第三点的参数，也就是`owner`则是`A`的类对象）

以下是一个描述器的实例，使用了上面已经定义好的`Descriptor`：

In [28]:
class DesTest:
    def __init__(self, name):
        self._name = name
    name = Descriptor()
obj = DesTest("Smith")
print(obj.name)
obj.name = "David"
del obj.name

This is __get__:  <__main__.Descriptor object at 0x0000016C63E61828> <__main__.DesTest object at 0x0000016C63E61940> <class '__main__.DesTest'>
Smith
This is __set__:  <__main__.Descriptor object at 0x0000016C63E61828> <__main__.DesTest object at 0x0000016C63E61940> David
This is __delete__:  <__main__.Descriptor object at 0x0000016C63E61828> <__main__.DesTest object at 0x0000016C63E61940>


如上所示，就是描述器的一般用法。当然，描述器也可以嵌套定义在目标类的内部。即：

In [29]:
class DesTest:
    def __init__(self, name):
        self._name = name
    class Descriptor:
        def __init__(self):#是可选的，也可以不定义构造函数
            pass
        def __get__(self, instance, owner):
            print("This is __get__: ", self, instance, owner)
            return instance._name
        def __set__(self, instance, value):
            print("This is __set__: ", self, instance, value)
            instance._name = value
        def __delete__(self, instance):
            print("This is __delete__: ", self, instance)
            del instance._name
    name = Descriptor()
obj = DesTest("Smith")
print(obj.name)
obj.name = "David"
del obj.name

This is __get__:  <__main__.DesTest.Descriptor object at 0x0000016C63E6EEB8> <__main__.DesTest object at 0x0000016C63E6EEF0> <class '__main__.DesTest'>
Smith
This is __set__:  <__main__.DesTest.Descriptor object at 0x0000016C63E6EEB8> <__main__.DesTest object at 0x0000016C63E6EEF0> David
This is __delete__:  <__main__.DesTest.Descriptor object at 0x0000016C63E6EEB8> <__main__.DesTest object at 0x0000016C63E6EEF0>


最后，看一个使用描述器实现`property`的例子：

In [33]:
class myProperty:
    def __init__(self, fget, fset, fdel, fdoc):
        self._fget = fget
        self._fset = fset
        self._fdel = fdel
        self.__doc__ = fdoc
    def __get__(self, instance, owner):
        print("This is __get__")
        return self._fget(instance)
    def __set__(self, instance, value):
        print("This is __set__")
        self._fset(instance, value)
    def __delete__(self, instance):
        print("This is __del__")
        self._fdel(instance)
class person:
    def __init__(self, name):
        self.__name = name
    def getName(self):
        print("This is getName")
        return self.__name
    def setName(self, value):
        print("This is setName")
        self.__name = value
    def delName(self):
        print("This is delName")
        del self.__name
    name = myProperty(getName, setName, delName, "This is name")
p = person("David")
print(p.name)
p.name = "Smith"
del p.name

This is __get__
This is getName
David
This is __set__
This is setName
This is __del__
This is delName


#### `__getattribute__`方法实现属性管理

该技术和`__getattr__`是相似的，唯二的区别如下：

1. `__getattr__`针对于未定义的属性进行拦截，包括在实例和类属性中都未定义的。但是`__getattribute__`则拦截一切属性，无论是否定义

2. `__getattr__`使用`__dict__`来避免死循环，而`__getattribute__`使用`object__getattribute__`来避免死循环。

### 装饰器

#### 函数装饰器

装饰器的作用在于跟踪函数或类的执行过程，在python中是一种非常强大的技术。首先来介绍函数装饰器，顾名思义就是针对函数的装饰，它有两种实现方式：一种是使用函数本身来实现装饰器，另一种是使用类来实现装饰器。最后，装饰器也分为带参数的装饰器和不带参数的装饰器。

* 使用函数实现的不带参数的装饰器

In [34]:
def decroator(func):
    print("This is in decroator")
    def wrapper(*args):
        for item in args:
            print(item)
    return wrapper
@decroator
def func(a, b, c):
    return a + b + c
func(1,2,3)

This is in decroator
1
2
3


如上面代码所示：函数装饰器实际上是拦截了函数的调用过程，具体而言实现了以下的过程：

```python
func = decroator(func)
```
上例子都是针对普通的函数而言的，在类内部的方法上也可以使用，只是要注意类方法的`self`函数的隐式传递：

In [47]:
def clazzDes_1(func):
    def wrapper(clazz, *args):
        print(clazz)
        for item in args:
            print(item)
        return func(clazz, *args)
    return wrapper

class DecrTest:
    def __init__(self, name):
        self._name = name
    @clazzDes_1
    def display_1(self):
        print("This is display_1: ", self._name)
obj = DecrTest("David")
obj.display_1()

This is display_1:  David


* 使用类实现的不带参数的装饰器

In [38]:
class decroator:
    def __init__(self, wrapper):
        self._wrapper = wrapper
    def __call__(self, *args):
        print("This is in decroator")
        for item in args:
            print(item)
        return self._wrapper(*args)
@decroator
def func(a,b,c):
    return a+b+c
print(func(1,2,3))

This is in decroator
1
2
3
6


和使用函数技术实现的装饰器一样，具体的过程就不再赘述。

**注意：类实现的装饰器无法用于装饰类方法**

#### 类装饰器

类装饰器顾名思义就是用来装饰类的，它和函数装饰器类似，同样也可以使用函数和类本身来实现：

* 使用函数实现的不带参数的类装饰器

In [57]:
def clazzDes(cls):
    class wrapper:
        def __init__(self, *args):
            self._wrapper = cls(*args)
            print(self._wrapper)
            for item in args:
                print(item)
        def __getattr__(self, name):
            return getattr(self._wrapper, name)
    return wrapper

@clazzDes
class testClass:
    def __init__(self, name):
        self._name = name
    def display(self):
        print("This is testClass: ", self._name)

obj = testClass("David")
obj.display()

David
This is testClass:  David


以上代码说明了，类装饰器和函数装饰器实际上是相似的，都是实现了以下过程：

```python
testClass = clazzDes(testClass)
obj = testClass("David")
```

* 使用类实现的不带参数的类装饰器

In [58]:
class clazzDes_1:
    def __init__(self, cls):
        self._cls = cls
    def __call__(self, *args):
        print(self, args)
        self._wrapper = self._cls(*args)
        return self
    def __getattr__(self, name):
        return getattr(self._wrapper, name)
@clazzDes_1
class testClass:
    def __init__(self, name):
        self._name = name
    def display(self):
        print("This is testClass: ", self._name)

obj = testClass("David")
obj.display()

<__main__.clazzDes_1 object at 0x0000016C63F21EF0> ('David',)
This is testClass:  David


#### 带参数的装饰器

In [68]:
def DesParameter(*args):
    print(args)
    def descriptors(func):
        def wrapper(*args):
            print(args)
            return func(*args)
        return wrapper
    return descriptors

@DesParameter("Hello,world")
def funcs(a, b, c):
    return a + b + c
funcs(1,2,3)

('Hello,world',)
(1, 2, 3)


6

上面的代码可以归结于以下调用形式：

```python
funcs = DesParameter("Hello,world")(funcs)
```

## 元类

元类在python中最简单的定义就是——type的子类。为了弄清楚这个模糊的定义，首先要弄清楚的是什么是type类，这对于后面的理解是非常重要的。type顾名思义就是“类型”的意思，它和python标准库中的其他类一样也是一个类，但是它也有非常特殊的地方。其他的类的实例就是一个普通的实例，而类型的实例是一个类。这是python3.0中一个更新的点，在2.X版本的python中类型的实例还不完全是一个类，在某些场景下它的实例也是一个普通的实例，不过今天我们要谈的不是2.X版本中的type。让我们用几句简单的代码来验证一下：

In [73]:
l = list()
l.__class__
type(l)

list

在代码中首先构造了一个list实例，然后用`type`的构造函数以`list`的实例为参数构造了一个`type`的实例，打印后我们发现`type`的实例是一个类，这和`list`实例的类（`l.__class__`）是相同的。这就印证了我们的一个论断——类型的实例就是类。

熟悉OOP的人特别是C++的人都应该了解，类只是声明而不是实例，只有在程序中实例化一个类才会分配内存。但是python中类也是一个叫做类对象的对象，它不是凭空出现在程序中的，它和实例一样也需要有代码对类对象进行实例化，说穿了类对象应该也是一种特殊的实例，为什么这么说呢？上面的代码应该能给我们些许暗示：通过`type`的构造函数构造出类型的实例是一个类，或者说类对象。

那么，写到这里我们就可以说`type`类负责类对象的创建，一般情况下这种创建是隐式的不被我们发觉的。而如果我们要显式地观察这个过程就可以通过创建元类来实现。在OOP中拦截一个类方法的方式之一就是继承这个类并且重载需要拦截的方法，那么这里就可以引出最开始给出的元类定义，元类就是`type`的子类。元类通过继承`type`类，进而重载`type`类中的一些方法来达到控制类对象生成的目的，这是元类编程中一个大体的思想。

类对象的创建过程和实例的创建过程相似或者说大体上是一致的，我们通过一个例子来了解吧。

In [75]:
class Meta(type):
    def __init__(self, *args):
        print("This is Meta's __init__: ", self, args)
        return type.__init__(self, *args)
    def __new__(self, *args):
        print("This is Meta's __new__: ", self, args)
        return type.__new__(self, *args)
class demo(metaclass=Meta):
    count = 0
    def __init__(self, name):
        self._name = name
    def method(self, age):
        print("This person {0}'s age is {1}".format(self._name, age))
obj = demo("David")
obj.method(25)

This is Meta's __new__:  <class '__main__.Meta'> ('demo', (), {'__module__': '__main__', '__qualname__': 'demo', 'method': <function demo.method at 0x0000016C63E4BD08>, 'count': 0, '__init__': <function demo.__init__ at 0x0000016C63E4BD90>})
This is Meta's __init__:  <class '__main__.demo'> ('demo', (), {'__module__': '__main__', '__qualname__': 'demo', 'method': <function demo.method at 0x0000016C63E4BD08>, 'count': 0, '__init__': <function demo.__init__ at 0x0000016C63E4BD90>})
This person David's age is 25


可见在类对象创建时首先调用元类中的`__new__`方法，然后调用`__init__`方法。其中`__new__`方法返回**类对象**的实例，`__init__`方法对类对象进行一些初始化。这两个方法都是`type`类中的方法，在这里我们继承`type`类后重载这两个方法等于覆盖了`type`类中的这两个方法。有了这个直观的感受我们接着进一步探索类对象的创建过程。

一个类在声明了元类之后（就是类名后面加个括号，里面写着`metaclass=XXX`）。当程序运行时，在`class`语句的末尾就会自动创建类对象。假设我们有一个`demo`类，声明它的元类为`Meta`，那么在`demo`的`class`语句完结后紧接着执行一句：

```python
demo=Meta(name,bases,dict)
```

传入三个参数，第一个是`demo`的类名称（字符串类型），第二个是`demo`类的父类元组，第三个是`demo`类的类字典。这时候就需要关注`Meta`了，`Meta`也是一个类，它是`type`的子类，同时`type`也是`Meta`的元类，就是说`Met`a类对象是`type`的实例。在`type`类中有一个`__call__`方法，这个方法是一个运算符重载方法，拦截`type(xxx,xxx,xxx,...)`这样的调用。回到刚才的

```python
demo=Meta(name,bases,dict)
```

由于`Meta`是`type`的实例，因此当这样的调用形式出现时必然会触发`type`中的`__call__`方法。由于`Meta`是`type`的实例，因此在传参的时候除了刚刚写出的三个参数外还会自动传入一个`Meta`自己，因此`type`的`__call__`方法实际上会接收到四个参数。现在程序运行到了`type`的`__call__`方法中，在这个方法中的调用过程我们用这样的一段伪代码来展示：

```python
class type:
    ...
    def __call__(self, className, bases, paraDict):
        ...
        clazz = self.__new__(self, className, bases, paraDict)
        obj = self.__init__(clazz, className, bases, paraDict)
        ...
        return clazz
    ...
```

正如同代码中所展示的，在`__call__`方法中首先调用元类的`__new__`方法得到一个类对象，再把这个类对象传入元类的`__init__`方法对这个类对象进行初始化最后再返回这个类对象，这也印证了最初我们的论断。元类构造一个类对象基本上就是这么一个过程。

为了更加说明元类构造类对象的过程，我从书上找了一个例子改了一下贴在这：

In [81]:
class SuperMeta(type):
    def __call__(self, className, bases, paraDict):
        print("This is SuperMeta __call__: ", self, className, bases, paraDict)
        clazz = self.__new__(self, className, bases, paraDict)
        obj = self.__init__(clazz, className, bases, paraDict)
        return clazz
    def __new__(self, className, bases, paraDict):
        print("This is SuperMeta __new__: ", self, className, bases, paraDict)
        return type.__new__(self, className, bases, paraDict)
    def __init__(self, className, bases, paraDict):
        print("This is SuperMeta __init__: ", self, className, bases, paraDict)
        return type.__init__(self, className, bases, paraDict)
class SubMeta(type, metaclass=SuperMeta):
    def __new__(self, className, bases, paraDict):
        print("This is SubMeta __new__: ", self, className, bases, paraDict)
        return type.__new__(self, className, bases, paraDict)
    def __init__(self, className, bases, paraDict):
        print("This is SubMeta __init__: ", self, className, bases, paraDict)
        return type.__init__(self, className, bases, paraDict)
class Eggs:
    pass
class Spam(Eggs, metaclass=SubMeta):
    data = 1
    def __init__(self, name):
        self._name = name
    def method(self, age):
        print("The person {0}'s age is {1}".format(self._name, age))
obj = Spam("David")
obj.method(25)

This is SuperMeta __new__:  <class '__main__.SuperMeta'> SubMeta (<class 'type'>,) {'__module__': '__main__', '__qualname__': 'SubMeta', '__init__': <function SubMeta.__init__ at 0x0000016C63E97D90>, '__new__': <function SubMeta.__new__ at 0x0000016C63E97C80>}
This is SuperMeta __init__:  <class '__main__.SubMeta'> SubMeta (<class 'type'>,) {'__module__': '__main__', '__qualname__': 'SubMeta', '__init__': <function SubMeta.__init__ at 0x0000016C63E97D90>, '__new__': <function SubMeta.__new__ at 0x0000016C63E97C80>}
This is SuperMeta __call__:  <class '__main__.SubMeta'> Spam (<class '__main__.Eggs'>,) {'__module__': '__main__', '__qualname__': 'Spam', 'data': 1, 'method': <function Spam.method at 0x0000016C63E4BC80>, '__init__': <function Spam.__init__ at 0x0000016C63E8A158>}
This is SubMeta __new__:  <class '__main__.SubMeta'> Spam (<class '__main__.Eggs'>,) {'__module__': '__main__', '__qualname__': 'Spam', 'data': 1, 'method': <function Spam.method at 0x0000016C63E4BC80>, '__init__'

这个例子是为了说明元类构建类对象时`__call__`方法的调用。首先梳理一下程序的结构：`Eggs`和`Spam`是两个常规的类，其中`Spam`是`Eggs`的子类，`Spam`的元类为`SubMeta`，而`SubMeta`的元类又为`SuperMeta`。之所以要绕这么一下就是为了让元类本身的构造过程也暴露出来。

接下来分析一下程序的运行。首先要构建`Spam`就要先构建`SubMeta`。`SubMeta`的元类为`SuperMeta`，在`SuperMeta`中定义了`__init__`和`__new__`方法，这是定义元类中一个比较常规的做法就不说了，我们要关注一下`SuperMeta`中定义的`__call__`方法并关注这个方法执行的时机，这很重要。首先，构建`SubMeta`等于执行这样的语句：

```python
SubMeta=SuperMeta(name,bases,dict)
```

但这是否意味着`SuperMeta`的`__call__`会执行呢？答案是否定的，**因为从原理上来说`SuperMeta`是`type`的实例**，因此上面的调用会执行`type`中的`__call__`方法而不是`SuperMeta`中的。接着，由于`SuperMeta`中重载了`type`的`__new__`和`__init__`方法，因此`type`类中的`__call__`会调用`superMeta`的这两个方法，调用之后`SubMeta`类对象就构建完成了，注意此时的`SubMeta`是`SuperMeta`的类实例，明白这点很重要。接下来就要构建`Spam`类对象：

```python
Spam=SubMeta(name,bases,dict)
```

由于`SubMeta`是`SuperMeta`的实例，因此上面代码的调用会触发`SuperMeta`的`__call__`方法，就是我们刚刚提到的那个。接着会调用`SubMeta`类中的`__init__`和`__new__`方法，如果`SubMeta`中这两个方法找不到或者没找全，程序就会顺着继承树找type类中的对应方法。

在这里，我们还可以继续思考一下。此处的`Spam`是`SubMeta`的类实例，如果在`SubMeta`中定义一个`__call__`方法，那么当`Spam`正常创建实例的时候会发生什么呢？这个问题就暗示了python创建实例背后的故事，其实这个过程和类对象的构建过程非常相似甚至代码都是一模一样的，同样也是触发`__call__`方法，进而调用`__new__`分配内存，然后调用`__init__`做一些初始化的工作。只不过和类对象不同的是类对象创建中`__new__`返回的是一个类，而实例创建中返回的是一个实例。之所以会有这样的区别在于在`__call__`方法中`__new__`和`__init__`的调用是取决于`__call__`的第一个参数，上面的例子中即为`Spam`，而`Spam`不是`type`的子类，因此`Spam`的`__init__`和`__new__`方法指向的是他自己或者是object的对应方法，因此才出现了`__new__`方法返回结果不同的现象。